<a href="https://colab.research.google.com/github/mkane968/Text-Mining-with-Student-Papers/blob/main/Uploading_and_Cleaning_Student_Essays_%26_Metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Upload Files and Add to Dataframe

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Selet all files to upload
from google.colab import files

uploaded = files.upload()

In [ ]:
#Add files into dataframe
import pandas as pd

essays = pd.DataFrame.from_dict(uploaded, orient='index')
essays.head()

In [ ]:
#Reset index and add column names to make wrangling easier
essays = essays.reset_index()
essays.columns = ["ID", "Text"]
essays

#Clean Texts

In [ ]:
#Remove encoding characters from Text column (b'\xef\xbb\xbf)
essays['Text'] = essays['Text'].apply(lambda x: x.decode('utf-8'))
essays.head()

In [ ]:
#Remove identifying information from ID
#Remove any occurences of "LATE_" from dataset (otherwise will skew ID cleaning)
essays['ID'] = essays['ID'].str.replace(r'LATE_', '', regex=True) 

#Split book on first underscore (_) in ID, keep only text in between first and second underscore (ID number)
start = essays["ID"].str.split("_", expand = True)
essays['ID'] = start[1]
essays['ID'] = essays['ID'].astype(int)
essays.head()

In [ ]:
#Remove newline characters
essays['Text'] = essays['Text'].str.replace(r'\s+|\\r', ' ', regex=True) 
essays['Text'] = essays['Text'].str.replace(r'\s+|\\n', ' ', regex=True) 
essays

In [ ]:
#Remove headers containing student name, instructor name, course name and date
#Split text on 2022 (will likely be last value in headers) and add all contents before to new column
headers = essays["Text"].str.split("2022", 1, expand = True)
essays['Header'] = headers[0]
print(essays['Header'])

#Add 2022 back to header column
essays['Header'] = essays['Header'] + '2022'
essays['Header'][0]

In [ ]:
#Remove any occurences of the header from the rest of the text in each cell (should be at top of each essay in portfolio)
essays['Text_NoHeaders'] = essays.apply(lambda row : row['Text'].replace(str(row['Header']), ''), axis=1)
essays['Text_NoHeaders'] 

In [ ]:
#Remove old text and header columns from dataframe 
essays = essays.drop(columns=['Text', 'Header'])
essays.head()

#Add Grades and Additional Metadata

In [ ]:
#Selet csv file to upload
from google.colab import files

uploaded = files.upload()

In [ ]:
#Create dataframe for metadata
metadata = pd.read_csv('2022-08-30T1337_Grades-LA-ENG-0802-711-10742-202220.csv')
metadata

In [ ]:
#Clean up metadata to prepare for merge
#Rename final portfolio column for clarity 
metadata = metadata.rename(columns={"Final Portfolio (1689777)": "Final Portfolio"})

#Choose which rows to keep (ID, section and final portfolio chosen here)
metadata = metadata[['ID', 'Section', "Final Portfolio"]]

#Drop first row (header row)-MAKE SURE TO ONLY RUN THIS ONCE!!
cleaned_metadata = metadata.iloc[1: , :]

#Drop decimal from ID (inconsistent with ID in essay dataframe)
cleaned_metadata['ID'] = cleaned_metadata['ID'].astype(int)

#Check cleaned DF
cleaned_metadata

In [ ]:
#Merge metadata and cleaned essays into new dataframe
#Will only keep rows where both essay and metadata are present
new_df = cleaned_metadata.merge(essays,on='ID')
new_df